In [1]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

/home/dic/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def conv_layer(input, channels_in, channels_out,name="conv"):
    with tf.name_scope(name):
        
        w=tf.Variable(tf.truncated_normal([5,5,channels_in,channels_out], stddev=0.1),name="W")
        #initializer=tf.glorot_uniform_initializer()
        #,initializer=tf.truncated_normal_initializer(stddev=0.05)
        b=tf.Variable(tf.constant(0.1, shape=[channels_out]), name="B")
        #,initializer=tf.random_uniform_initializer(-0.1,0.1)
        
        conv=tf.nn.conv2d(input,w, strides=[1,1,1,1], padding="SAME")
        act=tf.nn.relu(conv+b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

def dense_layer(input, channels_in, channels_out,name="dense"):
    with tf.name_scope(name):
        w=tf.Variable(tf.truncated_normal([channels_in,channels_out], stddev=0.1),name="W")
        b=tf.Variable(tf.constant(0.1, shape=[channels_out]), name="B")
        act=tf.nn.relu(tf.matmul(input, w)+b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

In [4]:
#setup placeholders
x= tf.placeholder(tf.float32, shape=[None,28,28,1],name="x")#here None will take input at the time of training and it will take the value equal to the batch size 
y= tf.placeholder(tf.float32, shape=[None,7],name="labels")


#Creating the network

#def conv_net(x):
conv1=conv_layer(x, 1, 32,"conv1")
drop1=tf.nn.dropout(conv1,keep_prob=0.1,name="dropout1")
flattened = tf.reshape(drop1, [-1, 28 * 28 * 32])

dense1=dense_layer(flattened,28*28*32,128,"dense1")
drop2=tf.nn.dropout(dense1,keep_prob=0.25,name="dropout2")
dense2=dense_layer(drop2,128,128,"dense2")
logits=dense_layer(dense2,128,7,"dense3")



In [5]:
#pred = conv_net(x)

#cross entropy as loss function
#cost=tf.reduce_mean=(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y))
with tf.name_scope("Loss"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
    tf.summary.scalar("Loss", cost)
#adamoptimizer
#optimizer=tf.train.AdamOptimizer(0.0002).minimize(cost)
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.0002).minimize(cost)
    
#computing accuracy
with tf.name_scope("accuracy"):
#Here you check whether the index of the maximum value of the predicted image is equal to the actual labelled image. and both will be a column vector.
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))

    #calculate accuracy across all the given images and average them out. 
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("Accuracy", accuracy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [6]:
import numpy as np
#import tensorflow.contrib.eager as tfe
# Load the training data into two NumPy arrays.

classes=7
features = np.load("X_2d.npy")
labels = np.load("y_onehot.npy")
X_train1, X_test1, y_train1, y_test1 = train_test_split(features, labels, test_size=0.2, random_state=1, shuffle=True)
print(y_train1.shape)
X_train = X_train1.reshape(-1, 28, 28, 1)
X_test= X_test1.reshape(-1,28,28,1)
y_train=y_train1.reshape(-1,7)
y_test=y_test1.reshape(-1,7)
print(X_train.shape)
print(y_train)
print(y_train.shape)


epoch=200
batch_size=200
steps=int(len(X_train)/batch_size)

(36504, 7)
(36504, 28, 28, 1)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
(36504, 7)


In [7]:

sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [8]:
writer_1 = tf.summary.FileWriter("/home/dic/jupyter/28x28dmdt/log_dir/train_1_200epochs_1")
#writer_2 = tf.summary.FileWriter("/home/dic/jupyter/28x28dmdt/log_dir/test_1_200epochs")
writer_1.add_graph(sess.graph)
write_op = tf.summary.merge_all()

In [9]:

train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []
#writer = tf.summary.FileWriter('/home/dic/jupyter/28x28dmdt/log_dir/1', sess.graph)    
    
for e in range(epoch):
    epoch_loss=0
    
    for batch in range(steps):
        #Calculate our current step
        step = e * steps + batch
    
        batch_x = X_train[batch*batch_size:min((batch+1)*batch_size,len(X_train))]
        batch_y = y_train[batch*batch_size:min((batch+1)*batch_size,len(y_train))]
        
            #batch,c=sess.run([optimizer,loss],feed_dict={x:batch_x,y=batch_y})
        
        opt = sess.run(optimizer, feed_dict={x: batch_x,y: batch_y})
        loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,y: batch_y})
        if step % 100 == 0:
            
            # Get Train Summary for one batch and add summary to TensorBoard                                                      
            summary = sess.run(write_op,feed_dict={x: batch_x,y: batch_y})
            writer_1.add_summary(summary, step)
            writer_1.flush()
    
    print("Iter " + str(e) + ", Loss= ",loss , ", Training Accuracy= ",acc)                                     
    print("Optimization Finished!")
        
       
    test_acc,valid_loss = sess.run([accuracy,cost], feed_dict={x: X_test,y : y_test})
    
    train_loss.append(loss)
    test_loss.append(valid_loss)
    train_accuracy.append(acc)
    test_accuracy.append(test_acc)
    #summary1 = sess.run(write_op,feed_dict={x: X_test,y : y_test})
    #writer_2.add_summary(summary1)
    #writer_2.flush()
    print("Testing Accuracy:",test_acc)
        #print("Testing Accuracy:","{:.5f}".format(test_acc))
        

Iter 0, Loss=  1.9535632 , Training Accuracy=  0.62
Optimization Finished!
Testing Accuracy: 0.62656146
Iter 1, Loss=  1.7061777 , Training Accuracy=  0.625
Optimization Finished!
Testing Accuracy: 0.6493535
Iter 2, Loss=  1.7118776 , Training Accuracy=  0.635
Optimization Finished!
Testing Accuracy: 0.6551611
Iter 3, Loss=  1.6217933 , Training Accuracy=  0.645
Optimization Finished!
Testing Accuracy: 0.65976334
Iter 4, Loss=  1.4778092 , Training Accuracy=  0.645
Optimization Finished!
Testing Accuracy: 0.662174
Iter 5, Loss=  1.4164929 , Training Accuracy=  0.64
Optimization Finished!
Testing Accuracy: 0.66294104
Iter 6, Loss=  1.3752381 , Training Accuracy=  0.645
Optimization Finished!
Testing Accuracy: 0.6640368
Iter 7, Loss=  1.364556 , Training Accuracy=  0.645
Optimization Finished!
Testing Accuracy: 0.6631602
Iter 8, Loss=  1.2972571 , Training Accuracy=  0.645
Optimization Finished!
Testing Accuracy: 0.66392726
Iter 9, Loss=  1.2981563 , Training Accuracy=  0.645
Optimizatio

Testing Accuracy: 0.73208416
Iter 79, Loss=  0.87164277 , Training Accuracy=  0.715
Optimization Finished!
Testing Accuracy: 0.7293447
Iter 80, Loss=  0.8332626 , Training Accuracy=  0.705
Optimization Finished!
Testing Accuracy: 0.72824895
Iter 81, Loss=  0.75853956 , Training Accuracy=  0.74
Optimization Finished!
Testing Accuracy: 0.7323033
Iter 82, Loss=  0.78958017 , Training Accuracy=  0.73
Optimization Finished!
Testing Accuracy: 0.73033094
Iter 83, Loss=  0.8378336 , Training Accuracy=  0.705
Optimization Finished!
Testing Accuracy: 0.73471403
Iter 84, Loss=  0.8572544 , Training Accuracy=  0.71
Optimization Finished!
Testing Accuracy: 0.72967345
Iter 85, Loss=  0.80998504 , Training Accuracy=  0.715
Optimization Finished!
Testing Accuracy: 0.73120755
Iter 86, Loss=  0.8177672 , Training Accuracy=  0.705
Optimization Finished!
Testing Accuracy: 0.7307692
Iter 87, Loss=  0.7605806 , Training Accuracy=  0.745
Optimization Finished!
Testing Accuracy: 0.7337278
Iter 88, Loss=  0.79

Iter 156, Loss=  0.7000431 , Training Accuracy=  0.77
Optimization Finished!
Testing Accuracy: 0.7648477
Iter 157, Loss=  0.69563127 , Training Accuracy=  0.73
Optimization Finished!
Testing Accuracy: 0.7648477
Iter 158, Loss=  0.6755744 , Training Accuracy=  0.77
Optimization Finished!
Testing Accuracy: 0.7671488
Iter 159, Loss=  0.68751246 , Training Accuracy=  0.77
Optimization Finished!
Testing Accuracy: 0.76846373
Iter 160, Loss=  0.70569944 , Training Accuracy=  0.785
Optimization Finished!
Testing Accuracy: 0.77021694
Iter 161, Loss=  0.7380208 , Training Accuracy=  0.775
Optimization Finished!
Testing Accuracy: 0.7718606
Iter 162, Loss=  0.7302742 , Training Accuracy=  0.75
Optimization Finished!
Testing Accuracy: 0.7686829
Iter 163, Loss=  0.7025138 , Training Accuracy=  0.78
Optimization Finished!
Testing Accuracy: 0.76912117
Iter 164, Loss=  0.6572162 , Training Accuracy=  0.805
Optimization Finished!
Testing Accuracy: 0.76594347
Iter 165, Loss=  0.7054357 , Training Accurac